In [1]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



dataset_train = CIFAR10(root='../data', train=True, 
            download=True, transform=transform)
dataset_test = CIFAR10(root='../data', train=False, 
             download=True, transform=transform)
train_loader = DataLoader(dataset_train, batch_size=32, 
                        shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, 
                       shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## Resnet 18

In [22]:
class ResBlock(nn.Module):
    expansion =1
    def __init__(self, in_channels, out_channels, stride=1, down_sample=False):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.stride = stride
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )
        
    def forward(self, x):
        shortcut=x
        print("shortcut: ",shortcut.shape)
        
        out=self.conv1(x)
        print(out.shape)
        out=self.bn1(out)
        print(out.shape)
        out=self.relu(out)
        print(out.shape)
        
        out=self.conv2(out)
        print(out.shape)
        out=self.bn2(out)
        print(out.shape)
        
        out=out+self.shortcut(out)
        out=self.relu(out)
        
        return out

In [56]:
class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = nn.ReLU(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = nn.ReLU(out)
        return out

In [57]:
class ResNet18(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet18, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, 2, stride=1)
        self.layer2 = self._make_layer(block, 128, 2, stride=2)
        self.layer3 = self._make_layer(block, 256, 2, stride=2)
        self.layer4 = self._make_layer(block, 512, 2, stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = nn.ReLU(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [23]:
class Resnet18(nn.Module):
    def __init__(self, block, num_classes=1000):
        super(Resnet18,self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)
        
        self.layer1_1 = ResBlock(in_channels=64, out_channels=64)
        self.layer1_2 = ResBlock(in_channels=64, out_channels=128, stride=1)
        
        self.layer2_1 = ResBlock(in_channels=128, out_channels=128)
        self.layer2_2 = ResBlock(in_channels=128, out_channels=256, stride=2)
        
        self.layer3_1 = ResBlock(in_channels=256, out_channels=256)
        self.layer3_2 = ResBlock(in_channels=256, out_channels=512, stride=2)
        
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * 10, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        print("resnet after maxpool :", x.shape)
        

        x = self.layer1_1(x)
        print(x.shape)
        x = self.layer1_2(x)
        print(x.shape)
        x = self.layer2_1(x)
        print(x.shape)
        x = self.layer2_2(x)
        print(x.shape)
        x = self.layer3_1(x)
        x = self.layer3_2(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
        
        

In [58]:
block
model=ResNet18(ResBlock, [2,2,2,2])

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

In [60]:
for i, (input_,target) in enumerate(test_loader):
    print(i, input_.shape, target.shape)
    break

0 torch.Size([32, 3, 224, 224]) torch.Size([32])


In [47]:
in_=torch.randn([10,64,32,32])
in_.shape

torch.Size([10, 64, 32, 32])

In [61]:

for i, (input_,target) in enumerate(train_loader):
    target = target.to(device)
    input_ = input_.to(device)
    print(target.shape, input_.shape)

    output = model(input_)
    loss = criterion(output,target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i%20 == 0):
        print("loss in epoch %d , step %d : %f" % (epoch, i,loss.data[0]))

torch.Size([32]) torch.Size([32, 3, 224, 224])


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not ReLU